### *This file acts as a source of information on **SimulaQron**, its **CLI**, **CQC**, **Twised Broker** and **NetQASM***

## Design 

### SimulaQron

What it does:
- SimulaQron is a tool for developing software applications running on the end nodes of a QI
- It simulates multiple QPUs held by the end nodes in single networks, as well as multiple connected networks, thus simulating QI
- It acts as a platform-dependant stand-in mimicking a real network of multiple systems and facilitating platform-independant development w/o Q-hardware
- Its main functionality is the simulation of qubit exchange and entanglement (w/o providing entanglement management functionalities)
- Its modular structural design enables the use of a multitude of quantum computer simulations to serve as a backend via Python
- The goal of SimulaQron is to realize QI based software applications which can later run and with real quantum hardware

What it does not:
- No entanglement management functionalities (still in development [20])
- No inherent security guarantees on simulated qubits
- Not a simulation for large scale quantum internet (testing quantum repeater schemes, error correcting codes, study the effects of noise on distant qubits etc.)
- No simulation of realistic noise and timings (for this, use NetSquit + SimulaQron)
- No security or authentification efforts for accessing the simulated QPUs

Challenges:
- Simulating entanglement classicly: for two entangled qubits *A* and *B* their state matrix must be virtually available to both nodes
- Qubits as a shared resource: Gates performed can create and destroy entanglement, the common challenges for concurrent access of resources in a network apply

### Virtual Qubits

![alt text](images/virtual_nodes_simulaqron.png)

- Each `virtualNode` can hold a number of `virtualQubit`s either simulated locally or remotely
- Each `virtualQubit` points to its corresponding `simulatedQubit`, which does not need to be on the same `virtualNode` or in the same network
- A `simulatedQubit` object is local to the node simulating it in its `quantumRegister`
- To marshall the mapping of virtual qubits **Twisted's Perspective Broker** is used
- The `quantumEngine` is the interface provided by the simulator to be made available as a backend for SimulaQron
- The `quantumEngine` functions as a `quantumRegister` addressing individual qubits by their position in the register
- Every `simulatedQubit` is manipulated directly w/o interacting with the `quantumRegister`
- Every `simulatedQubit` keeps track of its position in the register, thus preventing arbitrary growth of the register when removing and adding qubits to it (i.e. by measuring) 
- Two entangled `virtualQubit`s corresponding `simulatedQubit`s have to both be located within the same `quantumRegister`, requiring the `virtualNode` design
- The management behind `virtualQubit`s is client-server-based for every application using SimulaQron, giving full access to `virtualQubit`s in the system


### Register merges

- Centralized qubit management has lots of drawbacks -> distributed qubit management
- 'Each classical computer participating in the QI contributes its share to realize the overall simulation'
- **Problem:** Two `virtualQubit`s A and B have their `simulatedQubit`s on different nodes in the network, so an entangling operation on these qubits would require the `simulatedQubit`s to be moved to the same `quantumRegister` 
- **Solution:** transparent merging of registers (see img)
- For such merges, locking is required (SimulaQron currently uses simple and inefficient locking, which might be improved upon in the future)

![alt text](images/register_merging_simulaqron.png)

*Alice* and *Bob* initially share an *EPR-Pair* w/ both entangled `simulatedQubit`s in *Alice*'s `quantumRegister`.
*Bob* then performs a **CNOT** w/ one of his registers `simulatedQubit`s and one of *Alice*'s registers `simulatedQubit`s creating a *GHZ-state*.
The registers are merged and all three entangled qubits are located in *Alice*'s `quantumRegister` as `simulatedQubit`s while *Charlie* and *Bob* keep their references in the form of `virtualQubit`s.

## CQC

- **CQC** is a low level instruction set for quantum gates and measurements, combining classical and quantum communication.
- It enables applications to be programmed independant of the underlying platform, as long as the underlying platform provides the **CQC** interface
- Any language able to connect to the CQC server backend over a TCP connection can be used to develop using SimulaQron
- There is a CQC message format
- Requests may be given as single messages or batch jobs for complex operations
- CQC works w/ different message *types*
- Most basic message type: **COMMAND** (performs a specified command)
- CQC messages also contain meta data: CQC version, app id, length of msg, qubit id, ip, port of qubit dest node
- see table for options
- you can attach a list of commands for further exectution after the command has been executed or if a measurement outcome is of a certain value, which yields speed important for actual quantum networks concerning decoherence

![alt text](images/cqc_command_options.png)




### Message types

Message types from application to hardware/Simulaqron

![alt text](images/app2hardware_msgs_simulaqron.png)


Message types from hardware/Simulaqron to application

![alt text](images/hardware2app_msgs_simulaqron.png)

### Possible Commands

![alt text](images/cqc_commands.png)